# Stellar Velocity

In [1]:
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd
import numpy as np
from scipy.stats import binned_statistic
from binned_stats import get_binned_statistic

In [2]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup, set_axs_configuration
from auriga.paths import Paths
from auriga.parser import parse
from auriga.settings import Settings

In [3]:
figure_setup()

In [4]:
def read_data_circ_vel(simulation: str) -> tuple:
    """
    This method returns the radius in the disc plane and the tangential
    velocity of the stars in the main object.

    Parameters
    ----------
    simulation : str
        The simulation to consider.

    Returns
    -------
    tuple
        The properties.
    """
    s = Snapshot(simulation=simulation, loadonlytype=[4])
    s.add_extra_coordinates()

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    return (s.rho[is_real_star & is_main_obj],
            s.v_phi[is_real_star & is_main_obj],)

## Circular Velocity

In [5]:
# def add_panel_for_galaxy(simulation: str, ax: plt.Axes):
#     """
#     This method adds a panel with the results for a given simulation.

#     Parameters
#     ----------
#     simulation : str
#         The simulation.
#     ax : plt.Axes
#         The ax to which to add the plot.
#     """

#     rho, v_phi = read_data_circ_vel(simulation=simulation)
#     bin_edges = np.linspace(0, 40, 40 + 1)
#     mean_v_phi, std, median_v_phi, iqrange, bin_centers = get_binned_statistic(
#         x=rho,
#         values=v_phi,
#         bin_edges=bin_edges,
#         xrange=(0, 40)
#     )
#     ax.fill_between(x=bin_centers,
#                     y1=mean_v_phi - std,
#                     y2=mean_v_phi + std,
#                     color="tab:blue",
#                     alpha=0.15,
#                     linewidth=0)
#     ax.fill_between(x=bin_centers,
#                     y1=median_v_phi - iqrange,
#                     y2=median_v_phi + iqrange,
#                     color="tab:red",
#                     alpha=0.15,
#                     linewidth=0)
#     ax.plot(bin_centers, mean_v_phi, color="tab:blue")
#     ax.plot(bin_centers, median_v_phi, color="tab:red")
#     ax.text(x=0.05,
#             y=0.95,
#             s=r"$\texttt{" + simulation.upper() + "}$",
#             size=6.0, transform=ax.transAxes,
#             ha='left', va='top',
#             )

In [6]:
# def plot_circ_velocity_for_sample(simulations: list,
#                                   filename: str):
#     """
#     This method creates a plot of the circular velocity as a function of the
#     cylindrical radius r_{xy}.

#     Parameters
#     ----------
#     simulations : str
#         A list of simulations to plot.
#     filename : str
#         The name of the output file.
#     """

#     n_simulations = len(simulations)

#     fig = plt.figure(figsize=(7.2, 7.2))
#     gs = fig.add_gridspec(nrows=6, ncols=5, hspace=0.0, wspace=0.0)
#     axs = gs.subplots(sharex=True, sharey=True)

#     set_axs_configuration(
#         xlim=(0, 40), ylim=(0, 300),
#         xticks=[0, 10, 20, 30], yticks=[0, 100, 200],
#         xlabel=r'$r_{xy}$ [ckpc]',
#         ylabel=r"$v_\phi$ [$\mathrm{km} \, \mathrm{s}^{-1}$]",
#         axs=axs, n_used=n_simulations)

#     for idx, ax in enumerate(axs.flat):
#         if idx < n_simulations:
#             add_panel_for_galaxy(simulations[idx], ax=ax)
#         else:
#             ax.axis("off")

#     for ext in ["pdf", "png"]:
#         fig.savefig(
#             f"../images/stellar_circ_velocity/circ_velocity_{filename}.{ext}")
#     plt.close(fig)

In [7]:
# # Create plots for all galaxies in two figures
# settings = Settings()
# originals = [f"au{i}_or_l4_s127" for i in settings.galaxies]
# reruns = [f"au{i}_re_l4_s251" for i in settings.reruns]
# plot_circ_velocity_for_sample(simulations=originals, filename="originals")
# plot_circ_velocity_for_sample(simulations=reruns, filename="reruns")

## Velocities by Region

In [8]:
def read_data_vels(simulation: str) -> tuple:
    """
    This method returns the region tag, velocities and radius of the disc plane
    of the stars in the main object of the simulation.

    Parameters
    ----------
    simulation : str
        The simulation to consider.

    Returns
    -------
    tuple
        The properties.
    """
    s = Snapshot(simulation=simulation,
                 loadonlytype=[0, 1, 2, 3, 4, 5])
    s.add_circularity()
    s.add_reference_to_potential()
    s.add_normalized_potential()
    s.tag_particles_by_region(disc_std_circ=1.0,
                              disc_min_circ=0.4,
                              cold_disc_delta_circ=0.25,
                              bulge_max_specific_energy=-0.75)

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    v_z_signed = s.vel[:, 2] * np.sign(s.pos[:, 2])

    return (s.region_tag[is_real_star & is_main_obj],
            s.rho[is_real_star & is_main_obj],
            s.v_rho[is_real_star & is_main_obj],
            s.v_phi[is_real_star & is_main_obj],
            v_z_signed[is_real_star & is_main_obj],
            np.linalg.norm(s.vel, axis=1)[is_real_star & is_main_obj],)

In [9]:
def create_binned_dataframe(simulations: list, rxy_range: tuple,
                            n_bins: int) -> pd.DataFrame:
    settings = Settings()
    feats = []
    feat_names = []
    for simulation in simulations:
        region_tag, rho, v_rho, v_phi, vz, vel = read_data_vels(simulation)
        for component, tag in settings.component_tags.items():
            is_region = (region_tag == tag)

            v_rho_bind, bin_edges, _ = binned_statistic(x=rho[is_region],
                values=v_rho[is_region], bins=n_bins, range=rxy_range)

            v_z_bind, _, _ = binned_statistic(x=rho[is_region],
                values=vz[is_region], bins=n_bins, range=rxy_range)

            v_phi_bind, _, _ = binned_statistic(x=rho[is_region],
                values=v_phi[is_region], bins=n_bins, range=rxy_range)

            v_std_bind, _, _ = binned_statistic(x=rho[is_region],
                values=vel[is_region], statistic="std",
                bins=n_bins, range=rxy_range)

            feats.append(v_rho_bind)
            feat_names.append(
                f"{simulation.upper()}_RadialVelocity_{component}_km/s")
            feats.append(v_z_bind)
            feat_names.append(
                f"{simulation.upper()}_VerticalVelocity_{component}_km/s")
            feats.append(v_phi_bind)
            feat_names.append(
                f"{simulation.upper()}_TangentialVelocity_{component}_km/s")
            feats.append(v_std_bind)
            feat_names.append(
                f"{simulation.upper()}_VelocityDispersion_{component}_km/s")

    feats.append(bin_edges[1:] - np.diff(bin_edges)[0] / 2)
    feat_names.append("BinCenters_ckpc")

    df = pd.DataFrame(np.array(feats).T, columns=feat_names)

    return df

In [10]:
# Create DataFrame with data for all galaxies
settings = Settings()
originals = [f"au{i}_or_l4_s127" for i in settings.galaxies]
reruns = [f"au{i}_re_l4_s251" for i in settings.reruns]
galaxies = originals + reruns
data = create_binned_dataframe(
    simulations=galaxies, rxy_range=(1, 200), n_bins=199)

In [11]:
# for simulation in originals:
#     fig = plt.figure(figsize=(7.4, 2.0))
#     gs = fig.add_gridspec(nrows=1, ncols=4, hspace=0.0, wspace=0.0)
#     axs = gs.subplots(sharex=True, sharey=True)

#     for ax in axs.flat:
#         ax.grid(False)
#         ax.tick_params(which='both', direction="in")
#         ax.set_xlim(1, 200)
#         ax.set_ylim(-100, 350)
#         ax.set_xscale('log')
#         ax.set_xticks([1, 10, 100])
#         ax.set_yticks([-50, 0, 50, 100, 150, 200, 250, 300])
#         ax.set_xticklabels([1, 10, 100])
#         ax.set_xlabel(r'$r_{xy}$ [ckpc]')
#         ax.set_ylabel(r'Velocity [$\mathrm{km} \, \mathrm{s}^{-1}]$')
#         ax.label_outer()
#         ax.plot(ax.get_xlim(), [0] * 2, ls='--', lw=0.5, c="silver")

#     for component, idx in settings.component_tags.items():
#         is_finite = np.isfinite(
#             data[f"{simulation.upper()}_RadialVelocity_{component}_km/s"])
#         axs[idx].plot(
#             data["BinCenters_ckpc"][is_finite],
#             data[f"{simulation.upper()}_RadialVelocity_{component}_km/s"][
#                 is_finite],
#             lw=1.0, label="$v_r$", zorder=10, c="tab:green")

#         is_finite = np.isfinite(
#             data[f"{simulation.upper()}_VerticalVelocity_{component}_km/s"])
#         axs[idx].plot(
#             data["BinCenters_ckpc"][is_finite],
#             data[f"{simulation.upper()}_VerticalVelocity_{component}_km/s"][
#                 is_finite],
#             lw=1.0, label=r"$v^\dagger_z$", zorder=10, c="tab:red")

#         is_finite = np.isfinite(
#             data[f"{simulation.upper()}_TangentialVelocity_{component}_km/s"])
#         axs[idx].plot(
#             data["BinCenters_ckpc"][is_finite],
#             data[f"{simulation.upper()}_TangentialVelocity_{component}_km/s"][
#                 is_finite],
#             lw=1.0, label="$v_\phi$", zorder=10, c="tab:blue")

#         is_positive = \
#             data[
#                 f"{simulation.upper()}_VelocityDispersion_{component}_km/s"] \
#                     > 0
#         axs[idx].plot(
#             data["BinCenters_ckpc"][is_positive],
#             data[
#                 f"{simulation.upper()}_VelocityDispersion_{component}_km/s"][
#                     is_positive],
#             zorder=10, lw=1.0, label="$\sigma_v$", c="black")

#         axs[idx].text(
#             x=1.5, y=-80, size=8.0, ha='left', va='bottom',
#             s=settings.component_labels[settings.components[idx]])

#     axs[0].legend(loc="upper left", framealpha=0, fontsize=6.0)

#     axs[0].text(x=axs[0].get_xlim()[0],
#                 y=axs[0].get_ylim()[1],
#                 s=r"$\texttt{" + simulation.upper() + "}$",
#                 size=8.0,
#                 ha='left', va='bottom',
#                 )

#     fig.savefig(f"../images/stellar_velocities_by_region/{simulation}.pdf")
#     plt.close(fig)

## Average

In [21]:
# Remove zeros in standard deviation
for simulation in originals:
    for component, idx in settings.component_tags.items():
        data[f"{simulation.upper()}_VelocityDispersion_{component}_km/s"][
            data[f"{simulation.upper()}_VelocityDispersion_{component}_km/s"] < 0.1
        ] = np.nan

In [26]:
fig = plt.figure(figsize=(7.4, 8.0))
gs = fig.add_gridspec(nrows=4, ncols=4, hspace=0.0, wspace=0.0)
axs = gs.subplots(sharex=True, sharey=True)

for ax in axs.flat:
    ax.grid(False)
    ax.tick_params(which='both', direction="in")
    ax.set_xlim(1, 200)
    ax.set_ylim(-100, 350)
    ax.set_xscale('log')
    ax.set_xticks([1, 10, 100])
    ax.set_yticks([-50, 0, 50, 100, 150, 200, 250, 300])
    ax.set_xticklabels([1, 10, 100])
    ax.set_xlabel(r'$r_{xy}$ [ckpc]')
    ax.label_outer()
    ax.plot(ax.get_xlim(), [0] * 2, ls='--', lw=0.5, c="silver")

axs[0, 0].set_ylabel(r'$v_r$ [$\mathrm{km} \, \mathrm{s}^{-1}]$')
axs[1, 0].set_ylabel(r'$v_z^\dagger$ [$\mathrm{km} \, \mathrm{s}^{-1}]$')
axs[2, 0].set_ylabel(r'$v_\phi$ [$\mathrm{km} \, \mathrm{s}^{-1}]$')
axs[3, 0].set_ylabel(r'$\sigma_v$ [$\mathrm{km} \, \mathrm{s}^{-1}]$')

for simulation in originals:
    for component, idx in settings.component_tags.items():
        is_finite = np.isfinite(
            data[f"{simulation.upper()}_RadialVelocity_{component}_km/s"])
        axs[0, idx].plot(
            data["BinCenters_ckpc"][is_finite],
            data[f"{simulation.upper()}_RadialVelocity_{component}_km/s"][
                is_finite], lw=0.75, zorder=10, c="gainsboro")

        is_finite = np.isfinite(
            data[f"{simulation.upper()}_VerticalVelocity_{component}_km/s"])
        axs[1, idx].plot(
            data["BinCenters_ckpc"][is_finite],
            data[f"{simulation.upper()}_VerticalVelocity_{component}_km/s"][
                is_finite], lw=0.75, zorder=10, c="gainsboro")

        is_finite = np.isfinite(
            data[f"{simulation.upper()}_TangentialVelocity_{component}_km/s"])
        axs[2, idx].plot(
            data["BinCenters_ckpc"][is_finite],
            data[f"{simulation.upper()}_TangentialVelocity_{component}_km/s"][
                is_finite], lw=0.75, zorder=10, c="gainsboro")

        is_finite = np.isfinite(
            data[f"{simulation.upper()}_VelocityDispersion_{component}_km/s"])
        axs[3, idx].plot(
            data["BinCenters_ckpc"][is_finite],
            data[f"{simulation.upper()}_VelocityDispersion_{component}_km/s"][
                is_finite], lw=0.75, zorder=10, c="gainsboro")

features = ["RadialVelocity", "VerticalVelocity",
            "TangentialVelocity", "VelocityDispersion"]
for component, j in settings.component_tags.items():
    axs[0, j].text(
        x=0.5, y=0.95, size=8.0, ha="center", va="top",
        transform=axs[0, j].transAxes,
        s=settings.component_labels[settings.components[j]])
    for i, feature in enumerate(features):
        axs[i, j].plot(
            data["BinCenters_ckpc"],
            np.nanmean(data[[f"{simulation.upper()}_{feature}_{component}_km/s" for simulation in originals]], axis=1),
            lw=1.25, zorder=15, c=settings.component_colors[component])

fig.savefig(f"../images/stellar_velocities_by_region/originals_average.pdf")
plt.close(fig)

/u/fiza/conda-envs/auriga/lib/python3.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: Mean of empty slice
